In [ ]:
import sys  
sys.path.insert(0, '..')

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

from util.plots import show_plot

feature = pd.read_csv('../data/feature_nirspec100.csv').values
label = pd.read_csv('../data/label_nirspec100.csv').values

train_feature = feature[:50000]
train_label = label[:50000]
test_feature = feature[50000:]
test_label = label[50000:]


print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
print(train_feature.shape, train_label.shape)

In [2]:
def new_model(name):
    inputs = keras.Input(shape = (198,))
    
    x = layers.Dense(150, activation='relu')(inputs)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(44, activation='relu')(x)
    
    outputs = layers.Dense(12, activation='relu')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs, name=name)

In [ ]:
learning_rate = [0.0001, 0.001, 0.01]
optimizers = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adamax, keras.optimizers.Adadelta,
              keras.optimizers.Adagrad, keras.optimizers.Nadam, keras.optimizers.Ftrl]
losses = [keras.losses.MeanSquaredError(), keras.losses.MeanAbsoluteError(), keras.losses.MeanSquaredLogarithmicError()]

results = []
best_model = None
models = []
history_array = []

for lr in learning_rate:
    for opt in optimizers:
        optimizer = opt(learning_rate=lr)
        for loss in losses:
            model = new_model(f"{lr}_{optimizer._name}_{loss.name}_model")

            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=['mse']
            )
            model.summary()

            EPOCHS = 70
            history = model.fit(train_feature,
                                train_label,
                                epochs=EPOCHS, batch_size=200,
                                validation_split = 0.1)

            mse_final_result = min(history.history['val_mse'])
            model_copy = model
            if best_model is None or best_model[0] > mse_final_result:
                best_model = (mse_final_result, optimizer._name, loss.name, lr, model_copy)
            results.append((optimizer._name, loss.name, {'val_mse': mse_final_result, 'lr': lr,
                                                         'loss': (history.history['loss'][0], history.history['loss'][-1]),
                                                         'val_loss': (history.history['val_loss'][0], history.history['val_loss'][-1])}))
            models.append(model_copy)
            history_array.append(history)
        
print('-'*50)
print('BEST:', best_model[0:-1])
print(results)

In [ ]:
model = best_model[-1]
show_plot(model.predict(test_feature[:2500]), test_label[:2500],
          model.predict(train_feature[:5000]), train_label[:5000],
         (0.5, 0.5), 800)